### Import des librairies

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import regularizers
from PIL import Image
from keras.utils import to_categorical
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import numpy as np
import random
import joblib
from keras.utils import to_categorical
from sklearn import preprocessing
import tqdm
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras import layers
from keras.layers import Input, Lambda, Dense, Flatten
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

### Connexion au drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Récupération des datas et préparation du jeu de données

### Préparation du jeu de données

In [ ]:
# Chemin vers le dossier principal contenant les sous-dossiers

main_folder = '/content/drive/MyDrive/Detection_yeux/MMU-Iris-Database'

In [ ]:
def get_data_labels(main_folder, side_eye):

  """Fais un jeu de données sur MMU-Iris-Database avec en target "left" ou "right" """

  X = []
  y = []

  for target_folder in tqdm.tqdm(os.listdir(main_folder)):
    personnal_number_path = os.path.join(main_folder, target_folder)
    # chemin/42 etc
    if os.path.isdir(personnal_number_path):
      side_eye_path = os.path.join(personnal_number_path, side_eye)
      for eye_photo in os.listdir(side_eye_path): # Pour dossier dans dossier_gauche... (gauche)
        if eye_photo.lower().endswith(('.bmp', '.jpg', '.png')):
            image_path = os.path.join(side_eye_path, eye_photo)
            target = int(target_folder) # target = gauche ou droite
            image = cv2.imread(image_path)
            X.append(image)
            y.append(target)
  return X, y

X, y = get_data_labels(main_folder, "left")

### Enregistrement du jeu de données sur drive

In [ ]:
!pwd

In [ ]:
# Copie les variables d'entrée et de sortie sur drive pour pouvoir les charger plus rapidement à la relance du notebook

joblib.dump(X, "variables/X")
joblib.dump(y, "variables/y")
! cp variables/X /content/drive/MyDrive/Detection_yeux/variables/oeil_gauche/X
! cp variables/y /content/drive/MyDrive/Detection_yeux/variables/oeil_gauche/y

### Chargement du jeu de données

In [ ]:
# X, y = joblib.load("/content/drive/MyDrive/Detection_yeux/variables/oeil_gauche/X"), joblib.load("/content/drive/MyDrive/Detection_yeux/variables/oeil_gauche/y")

# En local
X = joblib.load(r"D:\vscode\files\cas_pratique_iris\variables\oeil_gauche\X")
y = joblib.load(r"D:\vscode\files\cas_pratique_iris\variables\oeil_gauche\y")


## Data augmentation

In [9]:
datagen = ImageDataGenerator(
    rotation_range = 15,
    height_shift_range = 0.2,
    fill_mode = "nearest",
)

In [ ]:
# X = np.array(X)
# labelenc = preprocessing.LabelEncoder()
# y = labelenc.fit_transform(y)

## Séparation en jeu d'entraînement, validation et test

In [10]:
y = np.array(y)
labelenc = LabelEncoder()
y_enc = labelenc.fit_transform(y)

In [50]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y_enc, test_size = 0.4, random_state = 42, stratify = y_enc)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 42, stratify = y_temp)

In [51]:
X_train.shape

(135, 240, 320, 3)

In [52]:
x = (3, 2)
y, u = X
print("y = " + y)

ValueError: too many values to unpack (expected 2)

In [44]:
IMG_SIZE = [240, 320]

def resize_and_rescale(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE[0], IMG_SIZE[1]])
  image = (image / 255.0)
  return image, label

def augment(image_label, seed):
  image, label = image_label
  image, label = resize_and_rescale(image, label)
  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)
  # Make a new seed.
  new_seed = tf.random.split(seed, num=1)[0, :]
  # Random crop back to the original size.
  image = tf.image.stateless_random_crop(
      image, size=[IMG_SIZE[0], IMG_SIZE[1], 3], seed=seed)
  # Random brightness.
  image = tf.image.stateless_random_brightness(
      image, max_delta=0.5, seed=new_seed)
  image = tf.clip_by_value(image, 0, 1)
  return image, label

numpy.ndarray

In [29]:
X_train = list(X_train)

In [48]:
len(y_train)

135

In [36]:
y_train = list(y_train)

In [40]:
type(X_train)

list

Traceback (most recent call last):
  File "C:\Users\geoff\AppData\Local\Temp\ipykernel_8316\2619649043.py", line 43, in <module>
    X_augmented, y_augmented = augment_dataset(X_train, y_train, 30)
  File "C:\Users\geoff\AppData\Local\Temp\ipykernel_8316\2619649043.py", line 20, in augment_dataset
    for batch in datagen.flow(img, label, batch_size=nb_aug_imgs):
  File "d:\vscode\virtualenv\lib\site-packages\keras\preprocessing\image.py", line 1545, in flow
    return NumpyArrayIterator(
  File "d:\vscode\virtualenv\lib\site-packages\keras\preprocessing\image.py", line 706, in __init__
    if y is not None and len(x) != len(y):
TypeError: object of type 'numpy.int64' has no len()



## Aperçu et vérification du jeu de donnée

In [ ]:
X.shape

In [ ]:
plt.imshow(X[0,:])
plt.show()
print(y[0])

In [ ]:
np.where(y_train == 11)

In [ ]:
fig = plt.figure(figsize=(10, 7))
rows = 2
columns = 2
fig.add_subplot(rows, columns, 1)
plt.imshow(X_train[12,:])
fig.add_subplot(rows, columns, 2)
plt.imshow(X_train[42,:])
fig.add_subplot(rows, columns, 3)
plt.imshow(X_train[65,:])
plt.show()

In [ ]:
print(np.where(y_val == 11))
print(np.where(y_test == 11))

In [ ]:
fig = plt.figure(figsize=(10, 7))
rows = 2
columns = 2
fig.add_subplot(rows, columns, 1)
plt.imshow(X_val[25,:])
fig.add_subplot(rows, columns, 2)
plt.imshow(X_test[24,:])
plt.show()

##Prétraitement des données

In [ ]:
X_train, X_val, X_test = np.array(X_train), np.array(X_val), np.array(X_test)
y_train, y_val, y_test = y_train.reshape(len(y_train), 1), y_val.reshape(len(y_val), 1), y_test.reshape(len(y_test), 1)
y_train_cat, y_val_cat, y_test_cat = to_categorical(y_train), to_categorical(y_val), to_categorical(y_test)

##Définition des modèles à tester

In [ ]:
NBCLASSES = len(np.unique(y_train))
IMSIZE=(240, 320, 3)

In [ ]:
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
early = EarlyStopping(monitor='val_loss',patience=15,restore_best_weights=True)

###Model 1

In [ ]:
# Chargement du modèle
vgg = VGG16(input_shape = IMSIZE, weights = 'imagenet', include_top = False)

# Freeze existing som VGG layers (before two lasts)
for layer in vgg.layers[:-2]:
    layer.trainable = False


model1 = tf.keras.Sequential([
  # Réajustement de la dimension de l'image
  layers.Resizing(IMSIZE[0], IMSIZE[1], crop_to_aspect_ratio=True),
  # Normalisation des pixels
  layers.Rescaling(1./255, input_shape=IMSIZE),
  # Ajout des couches VGG16
  vgg,
  # Ajout d'une couche de neurones
  # Conv2D(32, (3, 3), activation="relu", input_shape=IMSIZE),
  # MaxPooling2D((2, 2)),
  # Conv2D(32, (3, 3), activation="relu"),
  # MaxPooling2D((2, 2)),
  Flatten(),
  Dense(NBCLASSES, activation="softmax")
  ], name="left_side_classifier")


model1.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Montrer l'architecture du modèle et le nombre de paramètres entrainables
model1.build(input_shape=((None,) + IMSIZE))
model1.summary()

###Model 2

In [ ]:
# Même modèle mais avecsoftmax
# Chargement du modèle
vgg = VGG16(input_shape = IMSIZE, weights = 'imagenet', include_top = False)

# Freeze existing som VGG layers (before two lasts)
for layer in vgg.layers[:-2]:
    layer.trainable = False


model2 = tf.keras.Sequential([
  # Réajustement de la dimension de l'image
  layers.Resizing(IMSIZE[0], IMSIZE[1], crop_to_aspect_ratio=True),
  # Normalisation des pixels
  layers.Rescaling(1./255, input_shape=IMSIZE),
  # Ajout des couches VGG16
  vgg,
  # Ajout d'une couche de neurones
  # Conv2D(32, (3, 3), activation="relu", input_shape=IMSIZE),
  # MaxPooling2D((2, 2)),
  # Conv2D(32, (3, 3), activation="relu"),
  # MaxPooling2D((2, 2)),
  Flatten(),
  Dense(45, activation="softmax")
  ], name="model2")


model2.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Montrer l'architecture du modèle et le nombre de paramètres entrainables
model2.build(input_shape=((None,) + IMSIZE))
model2.summary()

###Modèle 3

In [ ]:
# Même modèle mais avecsoftmax
# Chargement du modèle
vgg = VGG16(input_shape = IMSIZE, weights = 'imagenet', include_top = False)

# Freeze existing som VGG layers (before two lasts)
for layer in vgg.layers[:-2]:
    layer.trainable = False


side_eye_classifier = tf.keras.Sequential([
  # Réajustement de la dimension de l'image
  layers.Resizing(IMSIZE[0], IMSIZE[1], crop_to_aspect_ratio=True),
  # Normalisation des pixels
  layers.Rescaling(1./255, input_shape=IMSIZE),
  # Ajout des couches VGG16
  vgg,
  # Ajout d'une couche de neurones
  Conv2D(32, (3, 3), activation="relu", padding="same"),
  Flatten(),
  Dense(45, activation="softmax")
  ], name="model3")


side_eye_classifier.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Montrer l'architecture du modèle et le nombre de paramètres entrainables
side_eye_classifier.build(input_shape=((None,) + IMSIZE))
side_eye_classifier.summary()

##Entraînement

In [ ]:
# tf.config.run_functions_eagerly(True)
model1.fit(X_train, y_train_cat, validation_data=(X_val, y_val_cat), batch_size = 50, epochs = 30000, callbacks = [rlrop, early])

##Prédiction

In [ ]:
# Exemple de données de test (remplacez cela par vos propres données)

y__pred = [np.argmax(v) for v in side_eye_classifier.predict(X_test)]  # Prédictions du modèle
y_test_metric = [np.argmax(v) for v in y_test]
y_test_metric

##Métriques

In [ ]:
import seaborn as sn
# Calcul de la matrice de confusion
conf_matrix = confusion_matrix(np.array(y_test_metric), np.array(y__pred))

# Calcul des mesures de performance
accuracy = accuracy_score(y_test_metric, y__pred)
precision = precision_score(y_test_metric, y__pred, average='macro')
recall = recall_score(y_test_metric, y__pred, average='macro')
f1 = f1_score(y_test_metric, y__pred, average='macro')

print("Matrice de confusion :")
sn.heatmap(conf_matrix, annot = True)
print("Précision : {:.2f}".format(precision))
print("Rappel : {:.2f}".format(recall))
print("Score F1 : {:.2f}".format(f1))
print("Exactitude : {:.2f}".format(accuracy))
print(conf_matrix)







##Sauv

In [ ]:
# side_eye_classifier.save("/content/drive/MyDrive/models.hdf5")

In [ ]:
labelenc.inverse_transform([42])